# Homework 2
#### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium

## <font color='purple'>Question 1</font> 
Go to the eurostat website and try to find a dataset that includes the european unemployment rates at a recent date.

Use this data to build a Choropleth map which shows the unemployment rate in Europe at a country level. Think about the colors you use, how you decided to split the intervals into data classes or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

We find the unemployment information under Population and social conditions > Labour Market > Employment and unemployment

In [ ]:
import pandas_datareader.data as pdr

df = pdr.DataReader("une_rt_a", 'eurostat')

In [37]:
test = pd.read_csv('lfsa_urgaed.tsv', sep='\t',encoding = 'utf8', header=[0])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [ ]:
from pandasdmx import Request
estat = Request('ESTAT', timeout)#lfsa_urgaed.smdx.xml
kiki = estat.datastructure('DSD_lfsa_urgaed').write(fromfreq=True)

In [3]:
#KEEPING ONLY STATS FOR THE MOST RECENT YEAR
df_eurostat = df.loc["2016-01-01"].reset_index(level=4, drop=True).unstack(-1).copy()

#KEEPING ONLY EUROPEAN COUNTRIES
df_eurostat = df_eurostat.drop(['Euro area (18 countries)', 'Euro area (19 countries)', 'Euro area (EA11-2000, EA12-2006, EA13-2007, EA15-2008, EA16-2010, EA17-2013, EA18-2014, EA19)', 'European Union (25 countries)', 'European Union (27 countries)',
       'European Union (28 countries)', 'Japan', 'United States'], axis=1)
#AND RENAME IT COUNTRY
df_eurostat.columns.name='Country'
df_eurostat

Country                                                      Austria  Belgium  \
UNIT                            AGE                 SEX                         
Percentage of active population From 25 to 74 years Females      4.9      6.7   
                                                    Males        5.6      7.0   
                                                    Total        5.3      6.8   
                                Less than 25 years  Females     10.2     18.2   
                                                    Males       12.1     21.7   
                                                    Total       11.2     20.1   
                                Total               Females      5.6      7.6   
                                                    Males        6.5      8.1   
                                                    Total        6.0      7.8   
Percentage of total population  From 25 to 74 years Females      3.1      4.0   
                                                    Males        4.2      4.9   
                                                    Total        3.7      4.5   
                                Less than 25 years  Females      5.6      4.7   
                                                    Males        7.3      6.7   
                                                    Total        6.5      5.7   
                                Total               Females      3.5      4.1   
                                                    Males        4.7      5.2   
                                                    Total        4.1      4.7   
Thousand persons                From 25 to 74 years Females     90.0    143.0   
                                                    Males      117.0    172.0   
                                                    Total      207.0    315.0   
                                Less than 25 years  Females     27.0     31.0   
                                                    Males       37.0     44.0   
                                                    Total       64.0     75.0   
                                Total               Females    117.0    173.0   
                                                    Males      153.0    216.0   
                                                    Total      270.0    390.0   

Country                                                      Bulgaria  \
UNIT                            AGE                 SEX                 
Percentage of active population From 25 to 74 years Females       6.5   
                                                    Males         7.6   
                                                    Total         7.1   
                                Less than 25 years  Females      16.9   
                                                    Males        17.4   
                                                    Total        17.2   
                                Total               Females       7.0   
                                                    Males         8.1   
                                                    Total         7.6   
Percentage of total population  From 25 to 74 years Females       3.8   
                                                    Males         5.3   
                                                    Total         4.5   
                                Less than 25 years  Females       3.3   
                                                    Males         4.9   
                                                    Total         4.1   
                                Total               Females       3.8   
                                                    Males         5.2   
                                                    Total         4.5   
Thousand persons                From 25 to 74 years Females      95.0   
                                                    Males       125.0   
                                            

In [4]:
dfES_pop = df_eurostat.copy()
# Keeping only the total in the sex index
dfES_pop.drop(['Females','Males'], level=-1, axis=0,inplace=True)
# Keeping only the total in the age index
dfES_pop.drop(['From 25 to 74 years', 'Less than 25 years'], level=-2, axis=0, inplace=True)
# Removing those indices (name and sex)
dfES_pop = dfES_pop.reset_index(level=[-1,-2], drop=True).transpose()
dfES_pop.reset_index(level=0, inplace=True)
dfES_pop.columns.name = ""

# Cleaning the Germany name
dfES_pop.Country = dfES_pop.Country.replace({'Germany (until 1990 former territory of the FRG)': 'Germany'})

# Display the results
dfES_pop

,Country,Percentage of active population,Percentage of total population,Thousand persons
0,Austria,6.0,4.1,270.0
1,Belgium,7.8,4.7,390.0
2,Bulgaria,7.6,4.5,247.0
3,Croatia,13.4,7.5,245.0
4,Cyprus,13.0,8.6,54.0
5,Czech Republic,4.0,2.6,212.0
6,Denmark,6.2,4.3,187.0
7,Estonia,6.8,4.8,47.0
8,Finland,8.8,5.8,237.0
9,France,10.1,6.1,2972.0


In [5]:
# Load the json data of the map
euro_geo_path = 'topojson/europe.topojson.json'
euro_geo = json.load(open(euro_geo_path))

In [ ]:
# Create a dictionary to convert country names to country IDs
dict_country_id = dict(map(lambda x : (x['properties']['NAME'],x['id']),euro_geo['objects']['europe']['geometries']))
# Adding a new column with country IDs
dfES_pop['Country ID'] = dfES_pop.Country.map(lambda x : dict_country_id[x])

In [ ]:
# Displaying the visualization map
m_euro = folium.Map([54,15], zoom_start=4)
m_euro.choropleth(
    geo_data = euro_geo,
    data=dfES_pop,
    columns=['Country ID', 'Percentage of total population'],
    key_on='feature.id',
    topojson='objects.europe',
    fill_color='OrRd', fill_opacity=0.5, line_opacity=0.2,
    legend_name="Percentage of unemployement in Europe"
    )
m_euro

## <font color='purple'>Question 2</font> 
Go to the amstat website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

HINT Go to the details tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through.
Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

## <font color='purple'>Question 3</font> 
Use the amstat website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between Swiss and foreign workers.

The Economic Secretary (SECO) releases a monthly report on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for foreign (5.1%) and Swiss (2.2%) workers.

Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (hint The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

## <font color='purple'>Question 4</font> 
BONUS: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in unemployment rates between the areas divided by the Röstigraben?